In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from itertools import product
import scipy.stats as ss
from dython.nominal import associations

In [2]:
df = pd.read_excel("Book3.xlsx") 
# importing first page as for the first question